In [ ]:
import pandas as pd

# URL del dataset (sostituiscilo con il tuo link)
url = "https://raw.githubusercontent.com/boradpreet/Weather_dataset/refs/heads/master/Weather%20Test%20Data.csv"
# Carica il dataset
df = pd.read_csv(url)

# 1. Rimozione dei valori mancanti
df = df.dropna()
# 2. Conversione della colonna 'RainToday' in valori numerici
df['RainToday'] = df['RainToday'].apply(lambda x: 1 if x == 'Yes' else 0)
# 3. Selezione delle feature 
features = ['MinTemp', 'MaxTemp', 'Humidity3pm', 'Humidity9am']
X = df[features]
y = df['RainToday']

from sklearn.model_selection import train_test_split
# Suddivisione dei dati (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Dimensioni del dataset di training: {len(X_train)}")
print(f"Dimensioni del dataset di test: {len(X_test)}")

from sklearn.ensemble import RandomForestClassifier
# Creazione e addestramento del modello
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, classification_report
# Predizioni sul set di test
y_pred = model.predict(X_test)
# Calcolo dell'accuratezza
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuratezza del modello: {accuracy:.2f}")
# Report di classificazione
print(classification_report(y_test, y_pred))

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
# Creiamo una heatmap con Seaborn
target_names = ["No Rain", "Rain"]
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
# Aggiungiamo i titoli
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
# Mostriamo il grafico
plt.show()

In [ ]:
import mlflow
import mlflow.sklearn

# Impostiamo un nome per l'esperimento
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("weather_classification_experiment")

In [ ]:
# Usando il modello creato nella Parte 1
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

n_estimators = 100

# Creiamo una nuova run MLflow
with mlflow.start_run():
    # Log Param
    mlflow.log_param("n_estimators", n_estimators)

    # Creazione del modello
    rf_model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf_model.fit(X_train, y_train)

    # Calcolo metriche
    y_pred = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    # Salviamo il modello
    mlflow.sklearn.log_model(rf_model, "random_forest_model")

    print(f"Esperimento concluso. Accuratezza registrata: {accuracy:.2f}")

In [ ]:
import mlflow.sklearn
from sklearn.metrics import accuracy_score

# Inserisci un run_id reale che trovi nell'interfaccia MLflow
RUN_ID = "<run_id_della_tua_run>"

loaded_model = mlflow.sklearn.load_model(f"runs:/{RUN_ID}/random_forest_model")

# Verifichiamo l'accuratezza
y_loaded_pred = loaded_model.predict(X_test)
acc_loaded = accuracy_score(y_test, y_loaded_pred)
print(f"Accuratezza del modello caricato: {acc_loaded:.2f}")